# EcoPoints Data Analysis

In [1]:
import json
import pandas as pd
import pydeck as pdk
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'notebook'
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)

from dash import Dash, dcc, html, Input, Output

In [2]:
with open('data.json') as f:
    data = json.load(f)

## Data Preperation

In [3]:
timestamps = []
engine_load = []
engine_rpm = []
coolant_temp = []
altitude = []
gps_speed = []
vehicle_speed = []
armin_speed = []
latitude = []
longitude = []

for entry in data:
    timestamps.append(pd.to_datetime(entry.get('timestamp'), unit='ms'))
    
    entry_data = entry.get('data', {})
    engine_load.append(float(entry_data.get('Engine Load', '0').replace(',', '.')))
    engine_rpm.append(float(entry_data.get('Engine RPM', '0').replace(',', '.')))
    coolant_temp.append(float(entry_data.get('Engine Coolant Temperature', '0').replace(',', '.')))
    altitude.append(float(entry_data.get('Altitude', '0').replace(',', '.')))
    gps_speed.append(float(entry_data.get('Gps-Speed', '0').replace(',', '.')))
    vehicle_speed.append(float(entry_data.get('Vehicle Speed', '0').replace(',', '.')))
    armin_speed.append(float(entry_data.get('Armin-Speed', '0').replace(',', '.')))
    latitude.append(float(entry_data.get('Latitude', '0')))
    longitude.append(float(entry_data.get('Longitude', '0')))

df = pd.DataFrame({
    'timestamp': timestamps,
    'Engine Load': engine_load,
    'Engine RPM': engine_rpm,
    'Coolant Temperature': coolant_temp,
    'Altitude': altitude,
    'Gps Speed': gps_speed,
    'Vehicle Speed': vehicle_speed,
    'Armin Speed': armin_speed,
    'Latitude': latitude,
    'Longitude': longitude
})

In [4]:
df = df[:-1]  

In [5]:
df

,timestamp,Engine Load,Engine RPM,Coolant Temperature,Altitude,Gps Speed,Vehicle Speed,Armin Speed,Latitude,Longitude
0,2024-10-24 13:21:13.199,78.0,1764.0,98.0,533.262937,45.0,41.0,44.0,48.322290,14.256009
1,2024-10-24 13:21:16.865,79.2,2019.0,98.0,537.843367,56.0,50.0,58.0,48.322815,14.255925
2,2024-10-24 13:21:20.531,64.3,2542.0,97.0,542.902183,60.0,61.0,60.0,48.323429,14.255708
3,2024-10-24 13:21:24.177,0.0,1716.0,97.0,545.912065,55.0,57.0,56.0,48.323844,14.255458
4,2024-10-24 13:21:27.816,66.3,1532.0,97.0,548.228076,52.0,51.0,49.0,48.324335,14.255235
...,...,...,...,...,...,...,...,...,...,...
248,2024-10-24 13:36:44.135,53.3,870.0,81.0,664.936749,47.0,8.0,48.0,48.360319,14.239841
249,2024-10-24 13:36:47.955,16.9,1168.0,81.0,664.936749,47.0,8.0,48.0,48.360319,14.239841
250,2024-10-24 13:36:51.758,35.3,833.0,81.0,664.936749,47.0,2.0,48.0,48.360319,14.239841
251,2024-10-24 13:36:55.543,0.0,805.0,82.0,664.936749,47.0,0.0,48.0,48.360319,14.239841


In [6]:
print(df.head())

                timestamp  Engine Load  Engine RPM  Coolant Temperature  \
0 2024-10-24 13:21:13.199         78.0      1764.0                 98.0   
1 2024-10-24 13:21:16.865         79.2      2019.0                 98.0   
2 2024-10-24 13:21:20.531         64.3      2542.0                 97.0   
3 2024-10-24 13:21:24.177          0.0      1716.0                 97.0   
4 2024-10-24 13:21:27.816         66.3      1532.0                 97.0   

     Altitude  Gps Speed  Vehicle Speed  Armin Speed   Latitude  Longitude  
0  533.262937       45.0           41.0         44.0  48.322290  14.256009  
1  537.843367       56.0           50.0         58.0  48.322815  14.255925  
2  542.902183       60.0           61.0         60.0  48.323429  14.255708  
3  545.912065       55.0           57.0         56.0  48.323844  14.255458  
4  548.228076       52.0           51.0         49.0  48.324335  14.255235  


In [13]:
app = Dash(__name__)

app.layout = html.Div([
    html.H4('Vehicle Data Over Time'),
    dcc.Graph(id="graph", style={'height': '700px'}),
    dcc.Checklist(
        id="checklist",
        options=[
            {'label': 'Engine Load', 'value': 'Engine Load'},
            {'label': 'Engine RPM', 'value': 'Engine RPM'},
            {'label': 'Coolant Temperature', 'value': 'Coolant Temperature'},
            {'label': 'Altitude', 'value': 'Altitude'},
            {'label': 'Gps Speed', 'value': 'Gps Speed'},
            {'label': 'Vehicle Speed', 'value': 'Vehicle Speed'},
            {'label': 'Armin Speed', 'value': 'Armin Speed'}
        ],
        value=['Engine RPM'],
        inline=True
    ),
])

@app.callback(
    Output("graph", "figure"), 
    Input("checklist", "value"))
def update_line_chart(selected_metrics):
    fig = px.line(df, x='timestamp', y=selected_metrics)
    
    fig.update_layout(
        title="Selected Vehicle Metrics Over Time",
        xaxis_title="Timestamp",
        yaxis_title="Value",
        legend_title="Metrics",
        xaxis={
            'rangeslider': {'visible': True},
            'type': 'date', 
            'range': [df['timestamp'].min(), df['timestamp'].max()]
        }
    )
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)

## Driven Path

In [ ]:
path_data = [
    {"path": [[row['Longitude'], row['Latitude']] for _, row in df.iterrows()]} 
]
layer = pdk.Layer(
    "PathLayer",
    path_data,
    get_path='path',
    get_width=1,  
    get_color='[0, 128, 255]',  
    width_scale=10,
    pickable=True,
)
view_state = pdk.ViewState(
    latitude=df['Latitude'].mean() if not df['Latitude'].empty else 0.0, 
    longitude=df['Longitude'].mean() if not df['Longitude'].empty else 0.0, 
    zoom=12, 
    bearing=0,
    pitch=0,
)

deck = pdk.Deck(layers=[layer], initial_view_state=view_state)
deck.to_html("driven_route_map.html")

In [ ]:
path_data = df[['Longitude', 'Latitude']].values.tolist()

path_layer = pdk.Layer(
    "PathLayer",
    data=[path_data],
    get_path='path',  
    get_color='[255, 0, 0]',  
    width_scale=10,  
    width_min_pixels=1,  
    get_width=1  
)

# Define the column layer for elevation
column_layer = pdk.Layer(
    "ColumnLayer",
    df,
    get_position='[Longitude, Latitude]',
    get_elevation='Altitude',
    elevation_scale=0.1, 
    radius=5,
    get_fill_color='[255, 0, 0]',
    pickable=True,
)

view_state = pdk.ViewState(
    latitude=df['Latitude'].mean(),
    longitude=df['Longitude'].mean(),
    zoom=12,
    bearing=0,
    pitch=50,
)

deck = pdk.Deck(layers=[path_layer, column_layer], initial_view_state=view_state)
deck.to_html("3d_path_map_with_lines_and_elevation.html")

In [ ]:
df['Altitude Change'] = df['Altitude'].diff().fillna(0) 
df['Engine Load Change'] = df['Engine Load'].diff().fillna(0)

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df['timestamp'],
    y=df['Altitude Change'],
    mode='lines',
    name='Change in Altitude (meters)',
    line=dict(color='orange', width=2),
    marker=dict(size=5, symbol='circle'),
))
fig.add_trace(go.Scatter(
    x=df['timestamp'],
    y=df['Engine Load Change'],
    mode='lines',
    name='Change in Engine Load (%)',
    line=dict(color='blue', width=2),
    marker=dict(size=5, symbol='circle'),
))
fig.update_layout(
    title='Changes in Altitude and Engine Load Over Time',
    xaxis_title='Timestamp',
    yaxis_title='Change in Values',
    legend_title='Legend',
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
    hovermode='x unified'
)
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df['timestamp'],
    y=df['Engine RPM'],
    mode='lines',
    name='Engine RPM',
    line=dict(color='blue', width=2),
    marker=dict(size=5, symbol='circle'),
))
fig.add_trace(go.Scatter(
    x=df['timestamp'],
    y=df['Coolant Temperature'],
    mode='lines',
    name='Coolant Temperature (°C)',
    line=dict(color='orange', width=2),
    marker=dict(size=5, symbol='circle'),
))
fig.update_layout(
    title='Engine RPM and Coolant Temperature Over Time',
    xaxis_title='Timestamp',
    yaxis_title='Values',
    legend_title='Legend',
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
    hovermode='x unified'
)
fig.show()

In [ ]:
df['Engine RPM Normalized'] = (df['Engine RPM'] - df['Engine RPM'].min()) / (df['Engine RPM'].max() - df['Engine RPM'].min())
df['Coolant Temperature Normalized'] = (df['Coolant Temperature'] - df['Coolant Temperature'].min()) / (df['Coolant Temperature'].max() - df['Coolant Temperature'].min())

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df['timestamp'],
    y=df['Engine RPM Normalized'],
    mode='lines',
    name='Engine RPM',
    line=dict(color='blue', width=2),
    marker=dict(size=5, symbol='circle'),
))
fig.add_trace(go.Scatter(
    x=df['timestamp'],
    y=df['Coolant Temperature Normalized'],
    mode='lines',
    name='Coolant Temperature (°C)',
    line=dict(color='orange', width=2),
    marker=dict(size=5, symbol='circle'),
))
fig.update_layout(
    title='Engine RPM and Coolant Temperature Over Time',
    xaxis_title='Timestamp',
    yaxis_title='Values',
    legend_title='Legend',
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
    hovermode='x unified'
)
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df['timestamp'],
    y=df['Engine Load'],
    mode='lines+markers',
    name='Engine Load (%)',
    line=dict(color='blue', width=2),
    marker=dict(size=5, symbol='circle'),
))
fig.add_trace(go.Scatter(
    x=df['timestamp'],
    y=df['Engine RPM'],
    mode='lines',
    name='Engine RPM',
    line=dict(color='orange', width=2),
    marker=dict(size=5, symbol='circle'),
))
fig.update_layout(
    title='Engine RPM and Engine Load Over Time',
    xaxis_title='Timestamp',
    yaxis_title='Values',
    legend_title='Legend',
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
    hovermode='x unified'
)
fig.show()

In [ ]:
df['Engine Load Normalized'] = (df['Engine Load'] - df['Engine Load'].min()) / (df['Engine Load'].max() - df['Engine Load'].min())
df['Engine RPM Normalized'] = (df['Engine RPM'] - df['Engine RPM'].min()) / (df['Engine RPM'].max() - df['Engine RPM'].min())

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df['timestamp'],
    y=df['Engine Load Normalized'],
    mode='lines',
    name='Engine Load (Normalized)',
    line=dict(color='blue', width=2),
    marker=dict(size=5, symbol='circle'),
))
fig.add_trace(go.Scatter(
    x=df['timestamp'],
    y=df['Engine RPM Normalized'],
    mode='lines',
    name='Engine RPM (Normalized)',
    line=dict(color='orange', width=2),
    marker=dict(size=5, symbol='circle'),
))
fig.update_layout(
    title='Normalized Engine RPM and Engine Load Over Time',
    xaxis_title='Timestamp',
    yaxis_title='Normalized Values',
    legend_title='Legend',
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
    hovermode='x unified'
)
fig.show()

In [ ]:
speedsFig = go.Figure()
speedsFig.add_trace(go.Scatter(x=df['timestamp'], y=df['Gps Speed'], mode='lines', name='GPS Speed', line=dict(color='blue')))
speedsFig.add_trace(go.Scatter(x=df['timestamp'], y=df['Armin Speed'], mode='lines', name='Armin Speed', line=dict(color='green')))
speedsFig.add_trace(go.Scatter(x=df['timestamp'], y=df['Vehicle Speed'], mode='lines', name='Vehicle Speed', line=dict(color='orange')))
speedsFig.update_layout(title='Speed Comparison Over Time', xaxis_title='Time', yaxis_title='Speed (km/h)', xaxis=dict(tickformat='%Y-%m-%d %H:%M:%S'), width=800, height=400)
speedsFig.show()

In [ ]:
loadSpeedFig = go.Figure()
loadSpeedFig.add_trace(go.Scatter(x=df['timestamp'], y=df['Engine Load'], mode='lines', name='Engine Load', line=dict(color='blue', width=2)))
loadSpeedFig.add_trace(go.Scatter(x=df['timestamp'], y=df['Vehicle Speed'], mode='lines', name='Vehicle Speed', line=dict(color='orange', width=2)))
loadSpeedFig.update_layout(
    title='Engine Load and Vehicle Speed Over Time',
    xaxis_title='Timestamp',
    yaxis_title='Values',
    legend_title='Legend',
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
    hovermode='x unified'
)
loadSpeedFig.show()

In [ ]:
df['Normalized Engine Load'] = (df['Engine Load'] - df['Engine Load'].min()) / (df['Engine Load'].max() - df['Engine Load'].min())
df['Normalized Vehicle Speed'] = (df['Vehicle Speed'] - df['Vehicle Speed'].min()) / (df['Vehicle Speed'].max() - df['Vehicle Speed'].min())
df['Normalized Engine RPM'] = (df['Engine RPM'] - df['Engine RPM'].min()) / (df['Engine RPM'].max() - df['Engine RPM'].min())

fig = go.Figure()
fig.add_trace(go.Scatter(x=df['timestamp'], y=df['Normalized Engine Load'], mode='lines', name='Normalized Engine Load', line=dict(color='blue', width=2)))
fig.add_trace(go.Scatter(x=df['timestamp'], y=df['Normalized Vehicle Speed'], mode='lines', name='Normalized Vehicle Speed', line=dict(color='orange', width=2)))
fig.add_trace(go.Scatter(x=df['timestamp'], y=df['Normalized Engine RPM'], mode='lines', name='Normalized Engine RPM', line=dict(color='green', width=2)))
fig.update_layout(
    title='Normalized Engine Load, Vehicle Speed, and Engine RPM Over Time',
    xaxis_title='Timestamp',
    yaxis_title='Normalized Values (0 to 1)',
    legend_title='Legend',
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
    hovermode='x unified'
)
fig.show()